In [ ]:
import os, time, random, logging , datetime , cv2 , csv , subprocess , json , math
import numpy as np

from utils import globo , xdv , tfh5 , sinet



CFG_SINET = {
    
    'sinet_version': 'fsd-sinet-vgg42-tlpf_aps-1',
    
    'graph_filename' : os.path.join(globo.FSDSINET_PATH,"fsd-sinet-vgg42-tlpf_aps-1.pb"),
    'metadata_file'  : os.path.join(globo.FSDSINET_PATH,"fsd-sinet-vgg42-tlpf_aps-1.json"),
    
    'audio_fs_input':22050,
    'batchSize' : 64,
    'lastPatchMode': 'repeat',
    'patchHopSize' : 50,
    
    
    'anom_labels' : ["Alarm","Boom","Crowd","Dog","Drill","Explosion","Fire","Gunshot and gunfire","Hammer","Screaming","Screech",\
                    "Shatter","Shout","Siren","Slam","Squeak","Yell"],
    'anom_labels_i' : [4,18,51,59,65,72,78,92,94,145,146,147,148,152,154,161,198],
    
    'anom_labels2' : ["Boom","Explosion","Fire","Gunshot and gunfire","Screaming",\
                    "Shout","Siren","Yell"],
    'anom_labels_i2' : [18,72,78,92,147,148,152,198],
    
    'full_or_max' : 'max', #chose output to (timesteps,labels_total) ot (1,labels_total)
    'labels_total' : 200
    
}


In [ ]:
frame_intervals , time_intervals = xdv.get_frame_intervals_chuncked_from_test_bg()
data = xdv.divide_into_train_valdt_test(frame_intervals)

In [ ]:
train_intervals = sum(np.shape(data["train"][k][1])[0] for k in range(len(data["train"])))

In [ ]:
full_train_fn = []

for root, dirs, files in os.walk(globo.SERVER_TRAIN_COPY_ALTER_PATH1):
    if root != globo.SERVER_TRAIN_COPY_ALTER_PATH1 + '/NN' : ## these are excluded from xdv train dataset
        print(root)
        for file in files:
            if file.find('.mp4') != -1:
                full_train_fn.append(os.path.join(root, file))
print(np.shape(full_train_fn))

In [ ]:
## to do 
## filter full_train_fn to only have normal videos

fixed_interval_size = 4 * 24  # 5 seconds * 24 fps

def get_random_fixed_interval(video_path, interval_size):
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        print(f"Error: Could not open the video file {video_path}")
        return None
    
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    
    if total_frames < interval_size:
        print(f"Error: Video {video_path} has fewer frames than the fixed interval size.")
        return None
    
    start_frame = random.randint(0, total_frames - interval_size)
    end_frame = start_frame + interval_size

    cap.release()

    return (start_frame, end_frame)

collected_intervals = []
total_intervals_collected = 0
interval_limit = 0

while total_intervals_collected < train_intervals and interval_limit < 1000:
    for video_path in full_train_fn:
        if total_intervals_collected >= train_intervals:
            break
        video_intervals = []
        num_intervals = random.randint(1, 10)
        for _ in range(num_intervals):
            interval = get_random_fixed_interval(video_path, fixed_interval_size)
            if interval is not None:
                video_intervals.append(interval)
                total_intervals_collected += 1
            if total_intervals_collected >= train_intervals:
                break
        collected_intervals.append((video_path, video_intervals))
        interval_limit += 1

print("Collected intervals:")
for index, item in enumerate(collected_intervals):
    print(f"{index + 1}. Video: {item[0]}, Intervals: {item[1]}")